In [1]:
using DataFrames, CSV
using JLD
using JuMP, Gurobi
using LinearAlgebra, Random, Printf, StatsBase, CategoricalArrays
using Plots, StatsPlots
using Distributions

In [2]:
const GRB_ENV = Gurobi.Env();

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-04


In [3]:
G_max = 74.0 #max gradient, prevent nerve stimulation by limiting max gradient and slew rate. mT/m
SR_max = 50.0 #max slew rate mT/m/ms

dt = 0.5 #ms
Teps = 26.4 #time to echo ms
T90 = 5.3 #90 degree RF time ms
T180 = 4.3 #half of 180 degree RF time ms
lambda_null = 60 #ms
epsilion = 1e-4
GAMMA = (42.58*10^3) * 2 * pi;


TEmin = T180 + 2 * Teps
TEmax = 97.0
bval = 500.0;

In [4]:
Teps = trunc(Int,Teps/dt+0.5)
T90 = trunc(Int, T90/dt + 0.5)
T180 = trunc(Int, T180/dt + 0.5)
TEmin = trunc(Int, TEmin/dt + 0.5)
TEmax = trunc(Int, TEmax/dt + 0.5)
lambda_null = lambda_null/dt
bval_target = sqrt(bval*2*TEmin / (GAMMA*GAMMA*0.001*dt))*0.50;

In [5]:
bval_target

0.02821966815168502

In [6]:
model = Model(() -> Gurobi.Optimizer(GRB_ENV))
set_optimizer_attributes(model, "TimeLimit" => 7200, "MIPGap" => 1e-4);

Set parameter TimeLimit to value 7200


In [7]:
#variable
@variable(model, TE>=0, Int) #Echo time
@variable(model, Tdiff>=0, Int) #Diffusion time
@variable(model, b>=0) #b_val
@variable(model, Gp1[1:TEmax]>=0) #Gradient positive part before 180 pulse
@variable(model, Gp2[1:TEmax]>=0) #Gradient positive part after 180 pulse
@variable(model, Gn1[1:TEmax]>=0) #Gradient negative part before 180 pulse
@variable(model, Gn2[1:TEmax]>=0) #Gradient negative part after 180 pulse
@variable(model, G_sign[1:TEmax], Bin) #Gradient sign (only positive or negative)
@variable(model, G_samp1[1:TEmax], Bin) #Gradient sampling (~T90 = 0)
@variable(model, G_samp2[1:TEmax], Bin) #Gradient sampling (TE/2-T180/2~TE/2+T180/2 = 0)
@variable(model, G_samp3[1:TEmax], Bin) #Gradient sampling (Tdiff~TE = 0)
@variable(model, G_inv[1:TEmax], Bin) #Gradient inversion (Gradient inversion after 180 pulse)
@variable(model, F[1:TEmax]) #F value
@variable(model, F_sign[1:TEmax], Bin) #F value
@variable(model, Fp[1:TEmax]>=0) #F value
@variable(model, Fn[1:TEmax]>=0) #F value

#objective
@objective(model, Min, TE)

#constraints
@constraint(model, TE_const, TEmin <= TE <= TEmax)
@constraint(model, Tdiff_cal, Tdiff == TE-Teps)

@constraint(model, T90_const1[i in 1:T90], G_samp1[i] == 1)
@constraint(model, T90_const2[i in T90+1:TEmax], G_samp1[i] == 0)

@constraint(model, T180_const1[i in 1:TEmax], i*G_samp2[i] <= TE/2+T180/2)
@constraint(model, T180_const2[i in 1:TEmax], (TEmax - (i-1))*G_samp2[i] <= TEmax - (TE/2-T180/2))
@constraint(model, T180_const3, sum(G_samp2[i] for i = 1:TEmax) >= T180)

@constraint(model, Tdiff_const[i in 1:TEmax], (TEmax-(i-1))*G_samp3[i] <= TEmax - (Tdiff-1))
@constraint(model, Tdiff_const2, sum(1-G_samp3[i] for i = 1:TEmax) <= Tdiff-1)

@constraint(model, Tinv_const[i in 1:TEmax], (TEmax-(i-1))*G_inv[i] <= TEmax - (TE/2-1))
@constraint(model, Tinv_const2, sum(1-G_inv[i] for i = 1:TEmax) <= TE/2)


@constraint(model, Gp_sampling_const1[i in 1:TEmax], Gp1[i] <= 10000*(1-G_samp1[i]-G_samp2[i]-G_samp3[i]))
@constraint(model, Gn_sampling_const1[i in 1:TEmax], Gn1[i] <= 10000*(1-G_samp1[i]-G_samp2[i]-G_samp3[i]))
@constraint(model, Gp_sampling_const2[i in 1:TEmax], Gp2[i] <= 10000*(1-G_samp1[i]-G_samp2[i]-G_samp3[i]))
@constraint(model, Gn_sampling_const2[i in 1:TEmax], Gn2[i] <= 10000*(1-G_samp1[i]-G_samp2[i]-G_samp3[i]))


@constraint(model, Gp_max_const1[i in 1:TEmax], Gp1[i] <= G_max*G_sign[i])
@constraint(model, Gn_max_const1[i in 1:TEmax], Gn1[i] <= G_max*(1-G_sign[i]))
@constraint(model, Gp_max_const2[i in 1:TEmax], Gp2[i] <= G_max*G_sign[i])
@constraint(model, Gn_max_const2[i in 1:TEmax], Gn2[i] <= G_max*(1-G_sign[i]))

@constraint(model, Gp1_const[i in 1:TEmax], Gp1[i] <= 10000 * (1-G_inv[i]))
@constraint(model, Gn1_const[i in 1:TEmax], Gn1[i] <= 10000 * (1-G_inv[i]))
@constraint(model, Gp2_const[i in 1:TEmax], Gp2[i] <= 10000 * G_inv[i])
@constraint(model, Gn2_const[i in 1:TEmax], Gn2[i] <= 10000 * G_inv[i])

@constraint(model, max_slew_const1[i in 2:TEmax], -SR_max*dt<= Gp1[i]-Gp1[i-1] <= SR_max*dt)
@constraint(model, max_slew_const2[i in 2:TEmax], -SR_max*dt<= Gn1[i]-Gn1[i-1] <= SR_max*dt)
@constraint(model, max_slew_const3[i in 2:TEmax], -SR_max*dt<= Gp2[i]-Gp2[i-1] <= SR_max*dt)
@constraint(model, max_slew_const4[i in 2:TEmax], -SR_max*dt<= Gn2[i]-Gn2[i-1] <= SR_max*dt)

@constraint(model, M0_const, sum(Gp1[i]-Gn1[i]-Gp2[i]+Gn2[i] for i = 1:TEmax) == 0)
@constraint(model, M1_const, sum(i*(Gp1[i]-Gn1[i]-Gp2[i]+Gn2[i]) for i = 1:TEmax) == 0)

@constraint(model, F_const1, F[1] == 0)
@constraint(model, F_const2[i in 2:TEmax], F[i] == sum(Gp1[j] - Gn1[j] - Gp2[j] + Gn2[j] for j=1:i)*0.001*0.001 *dt)
@constraint(model, F_const_abs1[i in 1:TEmax], Fp[i]-Fn[i] == F[i])
@constraint(model, F_const_abs2[i in 1:TEmax], Fp[i] <= 10 * F_sign[i])
@constraint(model, F_const_abs3[i in 1:TEmax], Fn[i] <= 10 * (1 - F_sign[i]))

@constraint(model, b_const, sum(Fp[i]+Fn[i] for i=1:TEmax) >= bval_target);
#@constraint(model, Eddy_const, ((Gp[Tdiff]-Gn[Tdiff])-(Gp[Tdiff-1]-Gn[Tdiff-1])) * eps(-Tdiff/lambda_null) <= epsilion);


In [8]:
solvetime = @elapsed optimize!(model)

Set parameter TimeLimit to value 7200
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 4854 rows, 3298 columns and 90006 nonzeros
Model fingerprint: 0x905e4bb4
Variable types: 2132 continuous, 1166 integer (1164 binary)
Coefficient statistics:
  Matrix range     [5e-07, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e+01, 2e+02]
  RHS range        [3e-02, 1e+04]
Presolve removed 2633 rows and 1590 columns
Presolve time: 0.07s
Presolved: 2221 rows, 1708 columns, 8012 nonzeros
Variable types: 1199 continuous, 509 integer (508 binary)

Root relaxation: objective 1.170000e+02, 240 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  117.00000    0    7          -  117.00000      -     -    0s
     0     0  117.0

81.700989667